In [1]:
# use the nolearn environment (source activate nolearn) to run this notebook
# Documentation: https://github.com/dnouri/nolearn/

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lasagne
from nolearn.lasagne import NeuralNet

%matplotlib inline

#### Eventually, make this into a function that reads the data, cleans it appropriately, shuffles and normalizes it, and splits it into training/test sets

In [2]:
# Read the training data for the neural network
# Input data is 250000x4 and output data is 250000x32
x = pd.read_csv('../parameters_250000.txt', sep=' ')
y = pd.read_csv('../results.txt', sep=' ', index_col=False)
# These functional groups do not exist in my model
y = y.drop(['light_aromatic_C-C', 'light_aromatic_methoxyl'], axis=1)

In [3]:
x

,Tmax[C],h[C/min],C[wt%],H[wt%]
0,520.849609,11606.079102,0.613126,0.062121
1,554.150391,11606.079102,0.613126,0.062121
2,520.849609,108867.651367,0.613126,0.062121
3,520.849609,11606.079102,0.593906,0.062121
4,520.849609,11606.079102,0.606840,0.064793
5,520.849609,108867.651367,0.600859,0.064793
6,554.150391,11606.079102,0.600859,0.064793
7,554.150391,108867.651367,0.606840,0.064793
8,554.150391,108867.651367,0.593906,0.062121
9,554.150391,108867.651367,0.600859,0.064793


I am going to follow the regression example in this tutorial to start figuring out nolearn:

http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/

For regression problems, I want the activation function for the output layer to be a rectified linear unit.

In [6]:
print x.shape, y.shape

(250000, 4) (250000, 30)


In [ ]:
net = NeuralNet(
    layers=[
        ('input', lasagne.layers.InputLayer),
        ('hidden', lasagne.layers.DenseLayer),
        ('output', lasagne.layers.DenseLayer),
        ]), 
    # layer parameters
    input_shape=()

In [4]:
NeuralNet?